In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sahilramane","key":"4367924c21eb72536366d68e85ed5dec"}'}

In [2]:
!pip install kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:19<00:00, 160MB/s]
100% 2.29G/2.29G [00:19<00:00, 124MB/s]


In [5]:
from zipfile import ZipFile
file_name='chest-xray-pneumonia.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [7]:
import tensorflow as tf

In [39]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)), # first conv #
      tf.keras.layers.MaxPooling2D(2, 2),

      tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # second conv #
      tf.keras.layers.MaxPooling2D(2,2),

      tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # third conv.#
      tf.keras.layers.MaxPooling2D(2,2), 

      # tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # fourth conv #
      # tf.keras.layers.MaxPooling2D(2,2),

      # tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # fifth conv.#
      # tf.keras.layers.MaxPooling2D(2,2),  

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(512, activation='relu'),

      tf.keras.layers.Dense(1, activation='sigmoid')

])

In [40]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6272)             

In [41]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [43]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

training_set_generator = train_datagen.flow_from_directory(
        'chest_xray/chest_xray/train',
        target_size=(128,128),
        batch_size=16 ,
        class_mode='binary')

val_set_generator = validation_datagen.flow_from_directory(
        'chest_xray/chest_xray/test',
        target_size=(128,128),
        batch_size=16,
        class_mode='binary')

history = model.fit(
      training_set_generator,
      steps_per_epoch=326,  
      epochs=4,
      validation_data = val_set_generator,
      validation_steps = 3)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/4
326/326 [==============================] - 164s 505ms/step - loss: 0.3474 - accuracy: 0.8439 - val_loss: 0.6445 - val_accuracy: 0.7292
Epoch 2/4
326/326 [==============================] - 163s 499ms/step - loss: 0.2277 - accuracy: 0.9103 - val_loss: 0.3967 - val_accuracy: 0.8125
Epoch 3/4
326/326 [==============================] - 163s 500ms/step - loss: 0.2163 - accuracy: 0.9135 - val_loss: 0.7037 - val_accuracy: 0.7292
Epoch 4/4
326/326 [==============================] - 162s 496ms/step - loss: 0.1799 - accuracy: 0.9331 - val_loss: 0.4499 - val_accuracy: 0.8125


In [60]:
test_image = image.load_img('normal.jpeg', target_size = (128, 128))

test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0)

result = model.predict_classes(test_image)
if result == [[0]]:
  print("result is NORMAL")
else: 
  print("Pneumonia Detected")

result is NORMAL
